In [1]:
import trading_gym
import numpy as np
from trading_gym.registry.gaia.v8.env import GAIAPredictorsContinuousV8

from datetime import datetime
from collections import namedtuple
import json
import os
import pandas as pd
import ray
from ray import rllib, tune
print(datetime.now())
print(trading_gym.__name__, trading_gym.__version__)
print(ray.__name__, ray.__version__)

from trading_gym.ray.logger import calculate_tearsheet, CustomLogger
from copy import deepcopy
# ray.init(num_cpus=8,ignore_reinit_error=True,object_store_memory= 10*100 )
ray.init(ignore_reinit_error=True)
#          object_store_memory = 50000000)

2019-08-21 13:47:40,445	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-21_13-47-40_444143_80401/logs.
2019-08-21 13:47:40,565	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:30353 to respond...


2019-08-21 13:47:40.439653
trading_gym 0.8.1
ray 0.7.3


2019-08-21 13:47:40,687	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:56401 to respond...
2019-08-21 13:47:40,690	INFO services.py:809 -- Starting Redis shard with 10.0 GB max memory.
2019-08-21 13:47:40,721	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-21_13-47-40_444143_80401/logs.
2019-08-21 13:47:40,723	WARNING services.py:1301 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-08-21 13:47:40,725	INFO services.py:1475 -- Starting the Plasma object store with 20.0 GB memory using /dev/shm.


{'node_ip_address': '10.0.5.4',
 'redis_address': '10.0.5.4:30353',
 'object_store_address': '/tmp/ray/session_2019-08-21_13-47-40_444143_80401/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-08-21_13-47-40_444143_80401/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-08-21_13-47-40_444143_80401'}

In [3]:
from ray import cloudpickle
from ray.utils import binary_to_hex, hex_to_binary


def cloudpickleloads(obj):
    if isinstance(obj, dict):
        try:
            return cloudpickle.loads(hex_to_binary(obj["value"]))
        except:
            for key, value in obj.items():
                if isinstance(value, dict):
                    if sorted(value) == ['_type', 'value']:
                        obj[key] = cloudpickle.loads(hex_to_binary(value["value"]))
                    else:
                        obj[key] = cloudpickleloads(value)
                elif isinstance(value, list):
                    for i, item in enumerate(value):
                        obj[key][i] = cloudpickleloads(item)
    return obj

In [4]:
paths = {2007: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2007/experiment_state-2019-07-08_14-26-15.json',
        2008: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2008/experiment_state-2019-07-08_14-52-38.json',
        2009: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2009/experiment_state-2019-07-08_15-20-13.json',
        2010: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2010/experiment_state-2019-07-08_15-47-57.json',
        2011: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2011/experiment_state-2019-07-08_16-15-37.json',
        2012: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2012/experiment_state-2019-07-08_16-44-41.json',
        2013: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2013/experiment_state-2019-07-08_17-13-04.json',
        2014: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2014/experiment_state-2019-07-08_17-39-13.json',
        2015: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2015/experiment_state-2019-07-08_18-06-10.json',
        2016: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2016/experiment_state-2019-07-08_18-31-47.json',
        2017: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2017/experiment_state-2019-07-08_18-58-52.json'
        }

In [5]:
for year,path in paths.items():
    with open(path) as f:
        metadata = json.load(f)

    runner_data = metadata['runner_data']
    stats = metadata['stats']

    checkpoint = metadata['checkpoints'][-1]
    checkpoint = cloudpickleloads(checkpoint)
    checkpoint_path = cloudpickle.loads(hex_to_binary(checkpoint['_checkpoint'])).value

    config = checkpoint['config']
    print(config)
    break

AttributeError: 'ETF' object has no attribute '_symbol'